<a href="https://colab.research.google.com/github/ABDULHAMEED-10/Food_Prediction_with_Nuterition_info/blob/main/Food_Recogniation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install torch torchvision Pillow flask requests

In [27]:
from flask import Flask, request, jsonify
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import requests
import os

In [29]:
MODEL_PATH = "/content/drive/MyDrive/Food Recogniation/resnet50model.pth"

model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 101)

try:
    checkpoint = torch.load(MODEL_PATH, map_location=torch.device("cpu"), weights_only=False)
    if "model_state" in checkpoint:
        model.load_state_dict(checkpoint["model_state"])
    elif "state_dict" in checkpoint:
        model.load_state_dict(checkpoint["state_dict"])
    else:
        model.load_state_dict(checkpoint)
    model.eval()
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)

Model loaded successfully!


In [30]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [31]:
class_names = [
    "apple_pie", "baby_back_ribs", "baklava", "beef_carpaccio", "beef_tartare",
    "beet_salad", "beignets", "bibimbap", "bread_pudding", "breakfast_burrito",
    "bruschetta", "caesar_salad", "cannoli", "caprese_salad", "carrot_cake",
    "ceviche", "cheesecake", "cheese_plate", "chicken_curry", "chicken_quesadilla",
    "chicken_wings", "chocolate_cake", "chocolate_mousse", "churros", "clam_chowder",
    "club_sandwich", "crab_cakes", "creme_brulee", "croque_madame", "cup_cakes",
    "deviled_eggs", "donuts", "dumplings", "edamame", "eggs_benedict",
    "escargots", "falafel", "filet_mignon", "fish_and_chips", "foie_gras",
    "french_fries", "french_onion_soup", "french_toast", "fried_calamari",
    "fried_rice", "frozen_yogurt", "garlic_bread", "gnocchi", "greek_salad",
    "grilled_cheese_sandwich", "grilled_salmon", "guacamole", "gyoza", "hamburger",
    "hot_and_sour_soup", "hot_dog", "huevos_rancheros", "hummus", "ice_cream",
    "lasagna", "lobster_bisque", "lobster_roll_sandwich", "macaroni_and_cheese",
    "macarons", "miso_soup", "mussels", "nachos", "omelette",
    "onion_rings", "oysters", "pad_thai", "paella", "pancakes",
    "panna_cotta", "peking_duck", "pho", "pizza", "pork_chop",
    "poutine", "prime_rib", "pulled_pork_sandwich", "ramen", "ravioli",
    "red_velvet_cake", "risotto", "samosa", "sashimi", "scallops",
    "seaweed_salad", "shrimp_and_grits", "spaghetti_bolognese", "spaghetti_carbonara",
    "spring_rolls", "steak", "strawberry_shortcake", "sushi", "tacos",
    "takoyaki", "tiramisu", "tuna_tartare", "waffles"
]

In [32]:
def predict_food(image_path):
    try:
        img = Image.open(image_path).convert("RGB")
        img = transform(img).unsqueeze(0)
        with torch.no_grad():
            outputs = model(img)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
        top5_prob, top5_classes = torch.topk(probabilities, 5)
        print("Top-5 Predictions:")
        for i in range(5):
            print(f"{class_names[top5_classes[0][i]]}: {top5_prob[0][i]:.4f}")
        return class_names[predicted.item()]
    except Exception as e:
        return f"Error in prediction: {e}"

In [33]:
def get_nutritional_info(food_name):
    api_key = 'iPfFkObG0Lc4NwBdM8l58Bt3nipiy8aNmMGjm1CQ'
    food_name = food_name.replace('_', ' ')
    fdc_id_url = f"https://api.nal.usda.gov/fdc/v1/food/{food_name}?api_key={api_key}"
    fdc_id_response = requests.get(fdc_id_url)
    if fdc_id_response.status_code == 200:
        fdc_id_data = fdc_id_response.json()
        if 'error' not in fdc_id_data:
            nutrients = {n['nutrientName']: n['value'] for n in fdc_id_data['foodNutrients']}
            return {
                'Food Name': fdc_id_data.get('description', 'N/A'),
                'Calories': nutrients.get('Energy', 'N/A'),
                'Protein': nutrients.get('Protein', 'N/A'),
                'Carbs': nutrients.get('Carbohydrate, by difference', 'N/A'),
                'Fats': nutrients.get('Total lipid (fat)', 'N/A')
            }
    search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={food_name}"
    search_response = requests.get(search_url)
    if search_response.status_code == 200:
        search_data = search_response.json()
        if search_data['foods']:
            food = search_data['foods'][0]
            nutrients = {n['nutrientName']: n['value'] for n in food['foodNutrients']}
            return {
                'Food Name': food.get('description', 'N/A'),
                'Calories': nutrients.get('Energy', 'N/A'),
                'Protein': nutrients.get('Protein', 'N/A'),
                'Carbs': nutrients.get('Carbohydrate, by difference', 'N/A'),
                'Fats': nutrients.get('Total lipid (fat)', 'N/A')
            }
        else:
            first_word = food_name.split()[0]
            search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={first_word}"
            search_response = requests.get(search_url)
            if search_response.status_code == 200:
                search_data = search_response.json()
                if search_data['foods']:
                    food = search_data['foods'][0]
                    nutrients = {n['nutrientName']: n['value'] for n in food['foodNutrients']}
                    return {
                        'Food Name': food.get('description', 'N/A'),
                        'Calories': nutrients.get('Energy', 'N/A'),
                        'Protein': nutrients.get('Protein', 'N/A'),
                        'Carbs': nutrients.get('Carbohydrate, by difference', 'N/A'),
                        'Fats': nutrients.get('Total lipid (fat)', 'N/A')
                    }
                else:
                    return {'error': 'Food not found'}
            else:
                return {'error': 'API request failed'}
    else:
        return {'error': 'API request failed'}

In [34]:
from google.colab import files

# Upload an image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Predict the food
food_name = predict_food(image_path)
print(f"\nPredicted Food: {food_name}")

# Get nutritional information
nutritional_info = get_nutritional_info(food_name)
if 'error' in nutritional_info:
    print(f"Error: {nutritional_info['error']}")
else:
    print("\nNutritional Information:")
    for key, value in nutritional_info.items():
        print(f"{key}: {value}")

Saving chicken.jpg to chicken.jpg
Top-5 Predictions:
chicken_wings: 0.3237
dumplings: 0.2179
frozen_yogurt: 0.0883
lasagna: 0.0770
gyoza: 0.0388

Predicted Food: chicken_wings

Nutritional Information:
Food Name: CHICKEN WINGS
Calories: 283
Protein: 18.6
Carbs: 0.0
Fats: 18.6
